<a href="https://colab.research.google.com/github/psyuktha/Reccomendation_system/blob/main/Movie_reccomendation_content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [2]:
!kaggle datasets download -d itschannel/telugu-movies-dataset

Dataset URL: https://www.kaggle.com/datasets/itschannel/telugu-movies-dataset
License(s): unknown
  0% 0.00/106k [00:00<?, ?B/s]
100% 106k/106k [00:00<00:00, 47.8MB/s]


In [3]:
!unzip telugu-movies-dataset.zip

Archive:  telugu-movies-dataset.zip
  inflating: TeluguMovies_dataset.csv  


In [4]:
df2=pd.read_csv('/content/TeluguMovies_dataset.csv')
df2.head()

,Unnamed: 0,Movie,Year,Certificate,Genre,Overview,Runtime,Rating,No.of.Ratings
0,0,Bahubali: The Beginning,2015.0,UA,"Action, Drama","In ancient India, an adventurous and darin...",159,8.1,99114
1,1,Baahubali 2: The Conclusion,2017.0,UA,"Action, Drama","When Shiva, the son of Bahubali, learns ab...",167,8.2,71458
2,2,1 - Nenokkadine,2014.0,UA,"Action, Thriller",A rock star must overcome his psychologica...,170,8.1,42372
3,3,Dhoom:3,2013.0,UA,"Action, Thriller","When Sahir, a circus entertainer trained i...",172,5.4,42112
4,4,Ra.One,2011.0,U,"Action, Adventure, Sci-Fi",When the titular antagonist of an action g...,156,4.6,37211


In [5]:
import pandas as pd
df=pd.read_csv('/content/TeluguMovies_dataset.csv',usecols=['Movie','Genre','Rating'])
df.head()

,Movie,Genre,Rating
0,Bahubali: The Beginning,"Action, Drama",8.1
1,Baahubali 2: The Conclusion,"Action, Drama",8.2
2,1 - Nenokkadine,"Action, Thriller",8.1
3,Dhoom:3,"Action, Thriller",5.4
4,Ra.One,"Action, Adventure, Sci-Fi",4.6


In [6]:

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

# Check if 'Genre' column contains any float values
if df['Genre'].apply(type).eq(float).any():
    print("Warning: 'Genre' column contains float values. Attempting to convert them to strings.")
    df['Genre'] = df['Genre'].astype(str)

# Transform the 'Genre' column
genre_vectors = mlb.fit_transform(df['Genre'].str.split('|'))

# Create a new DataFrame from the genre vectors
df_genre = pd.DataFrame(genre_vectors, columns=mlb.classes_)

# Concatenate the original DataFrame with the new genre DataFrame
df = pd.concat([df, df_genre], axis=1)

df.head()

,Movie,Genre,Rating,Action,"Action, Adventure","Action, Adventure, Comedy","Action, Adventure, Drama","Action, Adventure, Family","Action, Adventure, Fantasy","Action, Adventure, Romance",...,Mystery,"Mystery, Romance, Thriller","Mystery, Thriller",Romance,"Romance, Comedy, Drama","Romance, Drama","Romance, Thriller",Sport,Thriller,nan
0,Bahubali: The Beginning,"Action, Drama",8.1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Baahubali 2: The Conclusion,"Action, Drama",8.2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1 - Nenokkadine,"Action, Thriller",8.1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Dhoom:3,"Action, Thriller",5.4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ra.One,"Action, Adventure, Sci-Fi",4.6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df_genre.head()

,Action,"Action, Adventure","Action, Adventure, Comedy","Action, Adventure, Drama","Action, Adventure, Family","Action, Adventure, Fantasy","Action, Adventure, Romance","Action, Adventure, Sci-Fi","Action, Adventure, Thriller","Action, Adventure, Western",...,Mystery,"Mystery, Romance, Thriller","Mystery, Thriller",Romance,"Romance, Comedy, Drama","Romance, Drama","Romance, Thriller",Sport,Thriller,nan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(df_genre.values)
from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [11]:
query_index = np.random.choice(df_genre.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(df_genre.iloc[0,:].values.reshape(1, -1), n_neighbors = 6)



894


In [20]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(df.iloc[df_genre.index[query_index]][0:2]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, df.iloc[df_genre.index[indices.flatten()[i]]][0:2], distances.flatten()[i]))

Recommendations for Movie                                 Antham
Genre    Action, Crime, Thriller            
Name: 894, dtype: object:

1: Movie            Samarasimha Reddy
Genre    Action, Drama            
Name: 548, dtype: object, with distance of 0.0:
2: Movie    Rayalaseema Ramanna Chowdary
Genre       Action, Drama            
Name: 525, dtype: object, with distance of 0.0:
3: Movie          Parama Veera Chakra
Genre    Action, Drama            
Name: 547, dtype: object, with distance of 0.0:
4: Movie                  Narasimhudu
Genre    Action, Drama            
Name: 521, dtype: object, with distance of 0.0:
5: Movie            Nenu Naa Rakshasi
Genre    Action, Drama            
Name: 533, dtype: object, with distance of 0.0:


KeyError: 'rating'